In [11]:
from keras.utils import Sequence
from skimage.io import imread
from skimage.transform import resize
import numpy as np

class MY_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(imread(file_name), (28, 28))
               for file_name in batch_x]), np.array(batch_y)

In [48]:
from keras.utils import Sequence
class MY_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        image = [np.asarray(Image.open(file_name).resize([28,28])) for file_name in batch_x]
        image = image/np.amax(image)

        return np.array(image), np.array(batch_y)

In [55]:
106/20


5.3

In [36]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from pathlib import Path
import keras
import numpy as np
from PIL import Image

p = Path('./data/euc/') 
files = list(p.glob('**/*.png'))
files = str(files).split("'")
files = [ files[i] for i in list(range(1,len(files),2)) ]
classes = str(files).split('/')
classes = [ classes[i] for i in list(range(2,len(classes),3)) ]
classnames, indices = np.unique(classes, return_inverse=True)
labels = keras.utils.to_categorical(indices, len(np.unique(indices)))

In [56]:
my_training_batch_generator

In [54]:
batch_size = 106
num_epochs = 20
activation = "relu"
LR = 0.01
L2 = 0.001
num_training_samples = 106

my_training_batch_generator = MY_Generator(files, labels, batch_size)
#my_validation_batch_generator = My_Generator(validation_filenames, GT_validation, batch_size)

model = Sequential([
    Flatten(input_shape = (28, 28, 3), name = "flatten"),
    Dense(50, name = "hidden_1", kernel_regularizer=regularizers.l2(L2),),
    Activation(activation, name = "act_hidden_1"),
    Dense(2, name = "out"),
    Activation("softmax", name = "act_out")
    ])

sgd = optimizers.SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = sgd,
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

history = model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=(num_training_samples // batch_size),
                                          epochs=num_epochs,
                                          verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          use_multiprocessing=True,
                                          workers=16,
                                          max_queue_size=32)

Epoch 1/20

1/1 [==============================] - 2s 2s/step - loss: 0.7869 - acc: 0.5094
Epoch 2/20
1/1 [==============================] - 0s 7ms/step - loss: 0.7804 - acc: 0.5189
Epoch 3/20
1/1 [==============================] - 1s 535ms/step - loss: 0.7723 - acc: 0.5566
Epoch 4/20
1/1 [==============================] - 1s 518ms/step - loss: 0.7633 - acc: 0.6038
Epoch 5/20
1/1 [==============================] - 1s 538ms/step - loss: 0.7540 - acc: 0.6415
Epoch 6/20
1/1 [==============================] - 1s 536ms/step - loss: 0.7453 - acc: 0.6981
Epoch 7/20
1/1 [==============================] - 1s 539ms/step - loss: 0.7365 - acc: 0.7075
Epoch 8/20
1/1 [==============================] - 1s 545ms/step - loss: 0.7273 - acc: 0.7264
Epoch 9/20
1/1 [==============================] - 1s 549ms/step - loss: 0.7177 - acc: 0.7453
Epoch 10/20
1/1 [==============================] - 1s 538ms/step - loss: 0.7081 - acc: 0.7453
Epoch 11/20
1/1 [==============================] - 1s 551ms/step - loss: 

In [ ]:
def simple_NN(x_train, y_train, x_valid, y_valid, L2 = 0.001, activation = "relu", LR = 0.01, batch_size = 64):
    from keras import optimizers
    model = Sequential([
        Flatten(input_shape = (1,28, 28), name = "flatten"),
        Dense(50, name = "hidden_1", kernel_regularizer=regularizers.l2(L2),),
        Activation(activation, name = "act_hidden_1"),
        Dense(10, name = "out"),
        Activation("softmax", name = "act_out")
    ])
    sgd = optimizers.SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer = sgd,
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])
    checkpoints = keras.callbacks.ModelCheckpoint("./checkpoints/model_{epoch:02d}.hdf5", 
                                              monitor='val_loss', 
                                              verbose=0, 
                                              save_best_only=False, save_weights_only=False)
    history = model.fit(x_train, y_train, 
                    epochs = 10, batch_size = batch_size, 
                    callbacks = [checkpoints], 
                    validation_data = (x_valid, y_valid))
    return model, history

In [ ]:
from pathlib import Path
p = Path('./data/imgs/') 
classes = [x for x in p.iterdir() if x.is_dir()]
files = list(p.glob('**/*.jpg'))

In [ ]:
str(files[200000])

In [ ]:
# euc laptop
from pathlib import Path
p = Path('./data/') 
#classes = [x for x in p.iterdir() if x.is_dir()]
files = list(p.glob('**/*.png'))
classes = str(files).split('/')
classes = [ classes[i] for i in list(range(1,len(classes),2)) ]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(plt.imread(str(files[1])))
plt.show()

In [ ]:
from PIL import Image
im = Image.open(str(files[1])).resize([28,28])
#im = im.resize([28,50])
plt.imshow(im)
plt.show()

In [ ]:
import numpy as np
im = np.asarray(im)
im.shape

In [ ]:
import keras
keras.utils.to_categorical(classes, len(np.unique(classes)))

In [ ]:
classnames, indices = np.unique(classes, return_inverse=True)
keras.utils.to_categorical(indices, len(np.unique(indices)))

In [ ]:
files2[3]